In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model as lm

In [2]:
buildings = pd.read_csv('Houses_Sold.csv')
buildings.dropna(axis=0, inplace=True)

In [3]:
bb_freq = buildings.loc[(buildings['house_size'] <= 5000),].groupby(['bed','bath']).size().reset_index(name='count')
bb_freq = bb_freq.loc[(bb_freq['bed']) > (bb_freq['bath'])]
max_beds = np.min(bb_freq.loc[(bb_freq['count'] == 1),]['bed'])-1
print(f'maximum bedrooms are {max_beds}')
# max_baths = np.max(bb_freq['bath'])
# print(f'maximum bathrooms are {max_baths}')

maximum bedrooms are 12.0


In [ ]:
houses = buildings.loc[ (buildings['bed'] <= max_beds) & (buildings['bed'] > buildings['bath']) & (buildings['house_size'] <= 5000) & (buildings['house_size'] >= 1000) & (buildings['price'] > 1000) & (buildings['price'] < 10e6), ]

In [ ]:
houses['size_bins'] = np.round(houses['house_size'],-2)
price_var = houses.groupby('size_bins')['price'].var().reset_index(name='price_var')
price_var = price_var.sort_values(by='size_bins')
price_by_bin = houses.groupby('size_bins')['price'].apply(list)
df_price_by_bin = pd.DataFrame({str(bin_label): pd.Series(vals)
                                for bin_label, vals in price_by_bin.items()})
sqft = list(price_by_bin.index)
price_ar = df_price_by_bin.to_numpy()

fig, ax = plt.subplots(2,2, figsize=(10,5))
ax[0,0].scatter(houses['house_size'], houses['price'])
ax[0,0].set_ylabel('Price')
df_price_by_bin.boxplot(ax=ax[0,1])
ax[0,1].set_xticks([i for i in range(0, len(sqft)) if (i % 6 == 0) or (i == 0) or (i == len(sqft))])
ax[1,0].scatter(houses['house_size'], np.log(houses['price']))
ax[1,0].set_ylabel('ln(Price)')
ax[1,1].plot(price_var['size_bins'], price_var['price_var'])
ax[1,1].set_ylabel('Price Variance')
fig.supxlabel('House Square Footage')

In [ ]:
disaster = pd.read_csv('NRI_Table_Counties.csv')
disaster['NRI_ID'] = disaster['NRI_ID'].astype(str).str.replace(r'^C', '', regex=True)
disaster['NRI_ID'] = pd.to_numeric(disaster['NRI_ID'], errors='coerce').astype('Int64')  # nullable integer dtype

check = disaster.loc[disaster['COUNTY'] == 'Fulton', :]

In [ ]:
ZIPS = pd.read_csv('ZIP_COUNTY_062025.csv')

In [ ]:
dis_zip_county = disaster.merge(ZIPS[['ZIP', 'COUNTY', 'USPS_ZIP_PREF_CITY']], how='left', left_on='NRI_ID', right_on='COUNTY')
# There are 11 records (all in Conneticut and American Somoa) that do not have county information after the merge
check = dis_zip_county.loc[dis_zip_county['ZIP'].isna(),:]

In [ ]:
House_Disaster = houses.merge(dis_zip_county, how='left', left_on='zip_code', right_on='ZIP')

In [16]:
Income = pd.read_csv('22zpallagi.csv')
Income = Income.loc[Income['zipcode'] > 0, :]
Income = Income[['zipcode', 'N1', 'A00100']]
Income['Household_AGI'] = Income['A00100'] / Income['N1']

In [17]:
House_Income = houses.merge(Income, how='left', left_on='zip_code', right_on='zipcode')

In [18]:
zip_pop = pd.read_csv('zip_pop.csv')
zip_pop[['zcta_code','zip_code']] = zip_pop['Geographic Area Name'].str.split(' ', n=1, expand = True)
zip_pop['zip_code'] = pd.to_numeric(zip_pop['zip_code'], errors='coerce').astype('Int64')  # nullable integer dtype
zip_pop = zip_pop[['zip_code', ' Total Population']]

In [19]:
House_Income_Pop = House_Income.merge(zip_pop[['zip_code',' Total Population']], how='left', on='zip_code')

In [30]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_tweedie_deviance

In [ ]:
House_Income_Pop.dropna(axis=0, inplace=True)
X = House_Income_Pop[['bed', 'bath', 'house_size', 'zip_code', 'acre_lot', 'Household_AGI', ' Total Population']]
y = House_Income_Pop['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
XGReg = xgb_reg = xgb.XGBRegressor(
    objective='reg:tweedie',
    tweedie_variance_power=1.75, # Choose a value between 1 and 2 for overdispersed data
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
XGReg.fit(X_train, y_train)
y_pred = XGReg.predict(X_test)

In [40]:
# Evaluate using the Mean Tweedie Deviance
tweedie_deviance = mean_tweedie_deviance(y_test, y_pred, power=1.5)
print(f"Mean Tweedie Deviance: {tweedie_deviance:.4f}")

null_tweedie_deviance = mean_tweedie_deviance(y_test, [y_train.mean()]*len(y_test), power=1.5)
print(f"Null Model Mean Tweedie Deviance: {null_tweedie_deviance:.4f}")
print(f"Percent Deviance Explained: {(1-tweedie_deviance/null_tweedie_deviance)*100:.4f}")

Mean Tweedie Deviance: 175.4689
Null Model Mean Tweedie Deviance: 336.1751
Percent Deviance Explained: 47.8043
